# cjm-fasthtml-resources

> Resource monitoring and management system for tracking GPU/CPU usage and worker processes in FastHTML applications.

## Install

```bash
pip install cjm_fasthtml_resources
```

## Project Structure

```
nbs/
├── core/ (4)
│   ├── management_config.ipynb  # Configuration schema for resource management settings including GPU thresholds
│   ├── manager.ipynb            # Track worker processes and detect resource conflicts for GPU/CPU usage
│   ├── monitoring_config.ipynb  # Configuration schema for resource monitoring refresh intervals and SSE settings
│   └── validation.ipynb         # Validate resource availability before job execution and determine appropriate actions
└── utils/ (2)
    ├── plugin_utils.ipynb   # Utilities for analyzing plugin configurations and resource requirements
    └── route_helpers.ipynb  # Helper utilities for resource monitoring route handlers
```

Total: 6 notebooks across 2 directories

## Module Dependencies

```mermaid
graph LR
    core_management_config[core.management_config<br/>Management Configuration]
    core_manager[core.manager<br/>Resource Manager]
    core_monitoring_config[core.monitoring_config<br/>Monitoring Configuration]
    core_validation[core.validation<br/>Resource Validation]
    utils_plugin_utils[utils.plugin_utils<br/>Plugin Resource Utilities]
    utils_route_helpers[utils.route_helpers<br/>Route Helpers]

    core_validation --> core_manager
    utils_plugin_utils --> core_manager
```

*2 cross-module dependencies detected*

## CLI Reference

No CLI commands found in this project.

## Module Overview

Detailed documentation for each module in the project:

### Management Configuration (`management_config.ipynb`)
> Configuration schema for resource management settings including GPU thresholds

#### Import

```python
from cjm_fasthtml_resources.core.management_config import (
    RESOURCE_MANAGEMENT_SCHEMA
)
```
#### Variables

```python
RESOURCE_MANAGEMENT_SCHEMA = {7 items}
```

### Resource Manager (`manager.ipynb`)
> Track worker processes and detect resource conflicts for GPU/CPU usage

#### Import

```python
from cjm_fasthtml_resources.core.manager import (
    PLUGIN_RESOURCE_CONFIG_KEYS,
    ResourceType,
    ResourceStatus,
    ResourceConflict,
    WorkerState,
    ResourceManager
)
```
#### Classes

```python
class ResourceType(Enum):
    "Types of resources to monitor."
```

```python
class ResourceStatus(Enum):
    "Status of resource availability."
```

```python
@dataclass
class ResourceConflict:
    "Information about a resource conflict."
    
    resource_type: ResourceType
    status: ResourceStatus
    app_pids: List[int]  # PIDs from this application using the resource
    external_pids: List[int]  # External PIDs using the resource
    app_processes: List[Dict[str, Any]]  # Detailed info about app processes
    external_processes: List[Dict[str, Any]]  # Detailed info about external processes
```

```python
@dataclass
class WorkerState:
    """
    State of a worker process.
    
    Enhanced to support lifecycle-aware and cloud-aware plugins from cjm-fasthtml-plugins.
    """
    
    pid: int
    worker_type: str  # e.g., "transcription", "llm"
    job_id: Optional[str]
    plugin_id: Optional[str]
    plugin_name: Optional[str]
    loaded_plugin_resource: Optional[str]  # The plugin resource identifier currently loaded
    config: Optional[Dict[str, Any]]  # Current plugin configuration
    status: str = 'idle'  # idle, running, busy
    execution_mode: Optional[str]  # Execution mode (in_process, subprocess, cloud_gpu, etc.)
    child_pids: List[int] = field(...)  # PIDs of child processes
    container_id: Optional[str]  # Docker container ID if applicable
    conda_env: Optional[str]  # Conda environment name if applicable
    is_remote: bool = False  # Whether this worker uses remote/cloud resources
    remote_resource: Optional[Dict[str, Any]]  # Remote resource info (serialized RemoteResourceInfo)
```

```python
class ResourceManager:
    def __init__(self, gpu_memory_threshold_percent: float = 45.0):
        """
        Initialize the resource manager.
        
        Args:
            gpu_memory_threshold_percent: GPU memory usage threshold (default 45%)
                External processes using more than this percentage are considered conflicts
        """
        self._worker_states: Dict[int, WorkerState] = {}  # pid -> WorkerState
    """
    Manages resource tracking and conflict detection for the application.
    
    Tracks PIDs associated with application workers (transcription, LLM, etc.)
    and provides methods to check resource availability and conflicts.
    
    Enhanced to support lifecycle-aware and cloud-aware plugins from cjm-fasthtml-plugins.
    """
    
    def __init__(self, gpu_memory_threshold_percent: float = 45.0):
            """
            Initialize the resource manager.
            
            Args:
                gpu_memory_threshold_percent: GPU memory usage threshold (default 45%)
                    External processes using more than this percentage are considered conflicts
            """
            self._worker_states: Dict[int, WorkerState] = {}  # pid -> WorkerState
        "Initialize the resource manager.

Args:
    gpu_memory_threshold_percent: GPU memory usage threshold (default 45%)
        External processes using more than this percentage are considered conflicts"
    
    def register_worker(
            self,
            pid: int,
            worker_type: str,
            job_id: Optional[str] = None,
            plugin_id: Optional[str] = None,
            plugin_name: Optional[str] = None,
            loaded_plugin_resource: Optional[str] = None,
            config: Optional[Dict[str, Any]] = None,
            plugin_instance: Optional[Any] = None  # NEW: Optional plugin instance for lifecycle/cloud detection
        ) -> None
        "Register a worker process with the resource manager.

Args:
    pid: Process ID of the worker
    worker_type: Type of worker (e.g., "transcription", "llm")
    job_id: Optional job ID if worker is processing a job
    plugin_id: Optional plugin unique ID
    plugin_name: Optional plugin name
    loaded_plugin_resource: Optional identifier of the loaded plugin resource
    config: Optional plugin configuration
    plugin_instance: Optional plugin instance for lifecycle/cloud protocol detection"
    
    def get_all_related_pids(self, parent_pid: int) -> List[int]:
            """Get parent PID and all child PIDs managed by this worker.
            
            Args:
                parent_pid: Parent worker PID
            
            Returns:
                List of all PIDs (parent + children)
            """
            worker = self._worker_states.get(parent_pid)
            if not worker
        "Get parent PID and all child PIDs managed by this worker.

Args:
    parent_pid: Parent worker PID

Returns:
    List of all PIDs (parent + children)"
    
    def update_worker_state(
            self,
            pid: int,
            job_id: Optional[str] = None,
            plugin_id: Optional[str] = None,
            plugin_name: Optional[str] = None,
            loaded_plugin_resource: Optional[str] = None,
            config: Optional[Dict[str, Any]] = None,
            status: Optional[str] = None
        ) -> None
        "Update the state of a registered worker.

Args:
    pid: Process ID of the worker
    job_id: Optional job ID to update
    plugin_id: Optional plugin ID to update
    plugin_name: Optional plugin name to update
    loaded_plugin_resource: Optional loaded plugin resource to update
    config: Optional config to update
    status: Optional status to update"
    
    def unregister_worker(self, pid: int) -> None:
            """
            Unregister a worker process.
            
            Args:
                pid: Process ID of the worker to unregister
            """
            if pid in self._worker_states
        "Unregister a worker process.

Args:
    pid: Process ID of the worker to unregister"
    
    def get_worker_by_pid(self, pid: int) -> Optional[WorkerState]:
            """Get worker state by PID."""
            return self._worker_states.get(pid)
    
        def get_worker_by_job(self, job_id: str) -> Optional[WorkerState]
        "Get worker state by PID."
    
    def get_worker_by_job(self, job_id: str) -> Optional[WorkerState]:
            """Get worker state by job ID."""
            pid = self._job_to_pid.get(job_id)
            if pid
        "Get worker state by job ID."
    
    def get_all_workers(self) -> List[WorkerState]:
            """Get all registered workers."""
            return list(self._worker_states.values())
    
        def get_app_pids(self) -> Set[int]
        "Get all registered workers."
    
    def get_app_pids(self) -> Set[int]:
            """Get all PIDs managed by this application (parents only)."""
            return set(self._worker_states.keys())
        
        def get_all_app_pids_including_children(self) -> Set[int]
        "Get all PIDs managed by this application (parents only)."
    
    def get_all_app_pids_including_children(self) -> Set[int]:
            """Get all PIDs managed by this application including child processes.
            
            Returns:
                Set of all PIDs (parents and children)
            """
            all_pids = set(self._worker_states.keys())
            for worker in self._worker_states.values()
        "Get all PIDs managed by this application including child processes.

Returns:
    Set of all PIDs (parents and children)"
    
    def get_workers_by_type(self, worker_type: str) -> List[WorkerState]:
            """
            Get all workers of a specific type.
            
            Args:
                worker_type: Type of worker (e.g., "transcription", "llm", "ollama")
            
            Returns:
                List of workers matching the type
            """
            return [w for w in self._worker_states.values() if w.worker_type == worker_type]
    
        def get_active_worker_types(self) -> Set[str]
        "Get all workers of a specific type.

Args:
    worker_type: Type of worker (e.g., "transcription", "llm", "ollama")

Returns:
    List of workers matching the type"
    
    def get_active_worker_types(self) -> Set[str]:
            """
            Get set of all active worker types.
            
            Returns:
                Set of worker type strings
            """
            return {w.worker_type for w in self._worker_states.values()}
    
        def has_worker_type(self, worker_type: str) -> bool
        "Get set of all active worker types.

Returns:
    Set of worker type strings"
    
    def has_worker_type(self, worker_type: str) -> bool:
            """
            Check if a worker of the specified type exists.
            
            Args:
                worker_type: Type of worker to check
            
            Returns:
                True if at least one worker of this type exists
            """
            return any(w.worker_type == worker_type for w in self._worker_states.values())
        
        def get_cloud_workers(self) -> List[WorkerState]
        "Check if a worker of the specified type exists.

Args:
    worker_type: Type of worker to check

Returns:
    True if at least one worker of this type exists"
    
    def get_cloud_workers(self) -> List[WorkerState]:
            """Get all workers using cloud/remote resources.
            
            Returns:
                List of workers with is_remote=True
            """
            return [w for w in self._worker_states.values() if w.is_remote]
        
        def estimate_total_cloud_cost(self, duration_hours: float = 1.0) -> float
        "Get all workers using cloud/remote resources.

Returns:
    List of workers with is_remote=True"
    
    def estimate_total_cloud_cost(self, duration_hours: float = 1.0) -> float:
            """Estimate total cost of all running cloud resources.
            
            Args:
                duration_hours: Duration to estimate for (default 1 hour)
            
            Returns:
                Total estimated cost in USD
            """
            total = 0.0
            for worker in self.get_cloud_workers()
        "Estimate total cost of all running cloud resources.

Args:
    duration_hours: Duration to estimate for (default 1 hour)

Returns:
    Total estimated cost in USD"
    
    def check_gpu_availability(self) -> ResourceConflict:
            """
            Check GPU availability and identify conflicts.
            
            Uses configurable GPU memory threshold to determine if external processes
            are using significant GPU resources.
            
            Enhanced to detect child processes from lifecycle-aware plugins.
            
            Returns:
                ResourceConflict with details about GPU usage
            """
            try
        "Check GPU availability and identify conflicts.

Uses configurable GPU memory threshold to determine if external processes
are using significant GPU resources.

Enhanced to detect child processes from lifecycle-aware plugins.

Returns:
    ResourceConflict with details about GPU usage"
    
    def check_memory_availability(self, threshold_percent: float = 90.0) -> ResourceConflict:
            """
            Check system memory availability.
            
            Args:
                threshold_percent: Memory usage threshold to consider as conflict
            
            Returns:
                ResourceConflict with details about memory usage
            """
            try
        "Check system memory availability.

Args:
    threshold_percent: Memory usage threshold to consider as conflict

Returns:
    ResourceConflict with details about memory usage"
```

#### Variables

```python
PLUGIN_RESOURCE_CONFIG_KEYS = [5 items]
```

### Monitoring Configuration (`monitoring_config.ipynb`)
> Configuration schema for resource monitoring refresh intervals and SSE settings

#### Import

```python
from cjm_fasthtml_resources.core.monitoring_config import (
    RESOURCE_MONITOR_SCHEMA,
    LAST_UPDATE_TIMES,
    SSE_CONFIG
)
```
#### Variables

```python
RESOURCE_MONITOR_SCHEMA = {7 items}
LAST_UPDATE_TIMES = {7 items}
SSE_CONFIG = {3 items}
```

### Plugin Resource Utilities (`plugin_utils.ipynb`)
> Utilities for analyzing plugin configurations and resource requirements

#### Import

```python
from cjm_fasthtml_resources.utils.plugin_utils import (
    is_local_plugin,
    uses_gpu_device,
    get_plugin_resource_identifier,
    compare_plugin_resources,
    get_plugin_resource_requirements
)
```

#### Functions

```python
def is_local_plugin(plugin_meta) -> bool:
    """
    Check if a plugin is local (vs API-based).
    
    Args:
        plugin_meta: Plugin metadata with config_schema attribute
    
    Returns:
        True if plugin is local, False if API-based
    """
    if not hasattr(plugin_meta, 'config_schema') or not plugin_meta.config_schema
    """
    Check if a plugin is local (vs API-based).
    
    Args:
        plugin_meta: Plugin metadata with config_schema attribute
    
    Returns:
        True if plugin is local, False if API-based
    """
```

```python
def uses_gpu_device(plugin_config: Dict[str, Any]) -> bool:
    """
    Check if a plugin is configured to use GPU.
    
    Args:
        plugin_config: The plugin's current configuration
    
    Returns:
        True if plugin will use GPU, False otherwise
    """
    device = plugin_config.get('device', '').lower()

    # Check for explicit GPU indicators
    if device and device != 'cpu'
    """
    Check if a plugin is configured to use GPU.
    
    Args:
        plugin_config: The plugin's current configuration
    
    Returns:
        True if plugin will use GPU, False otherwise
    """
```

```python
def get_plugin_resource_identifier(plugin_config: Dict[str, Any]) -> Optional[str]:
    """
    Extract the plugin resource identifier from plugin configuration.
    
    Checks common plugin resource configuration keys like 'resource_id', 'model_id', 
    'model', 'model_name', etc.
    
    Args:
        plugin_config: The plugin's configuration
    
    Returns:
        Plugin resource identifier string, or None if not found
    """
    for key in PLUGIN_RESOURCE_CONFIG_KEYS
    """
    Extract the plugin resource identifier from plugin configuration.
    
    Checks common plugin resource configuration keys like 'resource_id', 'model_id', 
    'model', 'model_name', etc.
    
    Args:
        plugin_config: The plugin's configuration
    
    Returns:
        Plugin resource identifier string, or None if not found
    """
```

```python
def compare_plugin_resources(
    config1: Dict[str, Any],
    config2: Dict[str, Any]
) -> bool
    """
    Compare two plugin configurations to see if they use the same plugin resource.
    
    Args:
        config1: First plugin configuration
        config2: Second plugin configuration
    
    Returns:
        True if both configs specify the same plugin resource, False otherwise
    """
```

```python
def get_plugin_resource_requirements(
    plugin_id: str,
    plugin_registry,  # Plugin registry with get_plugin, load_plugin_config methods
    plugin_config: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]
    """
    Get resource requirements for a plugin.
    
    Args:
        plugin_id: Unique plugin ID
        plugin_registry: Plugin registry instance
        plugin_config: Optional plugin configuration
    
    Returns:
        Dictionary with resource requirement information:
        - is_local: Whether it's a local plugin
        - uses_gpu: Whether it uses GPU
        - plugin_resource: Resource identifier
        - device: Device configuration
    """
```


### Route Helpers (`route_helpers.ipynb`)
> Helper utilities for resource monitoring route handlers

#### Import

```python
from cjm_fasthtml_resources.utils.route_helpers import (
    wrap_card_in_container,
    create_card_update
)
```

#### Functions

```python
def wrap_card_in_container(content, html_id, card_cls=None, bg_cls=None, shadow_cls=None, **kwargs)
    """
    Wrap card content in a Div container with standard styling.
    
    This consolidates the common pattern of wrapping monitoring cards
    in styled containers.
    
    Args:
        content: Card content to wrap
        html_id: HTML ID for the container
        card_cls: Card class (optional, can be provided via DaisyUI)
        bg_cls: Background class (optional, can be provided via DaisyUI)
        shadow_cls: Shadow class (optional, can be provided via Tailwind)
        **kwargs: Additional attributes for the Div
    
    Returns:
        Div: Wrapped card container
    """
```

```python
def create_card_update(
    render_fn: Callable, 
    info: Dict[str, Any], 
    target_id: str, 
    swap_type: str = "outerHTML"
)
    """
    Create an OOB swap update for a card.
    
    This consolidates the pattern of creating OOB swaps for card updates
    in SSE streaming.
    
    Args:
        render_fn: Function to render the card
        info: Info dictionary to pass to render function
        target_id: Target HTML ID for the swap
        swap_type: Type of swap (default "outerHTML")
    
    Returns:
        OOB swap element
    """
```


### Resource Validation (`validation.ipynb`)
> Validate resource availability before job execution and determine appropriate actions

#### Import

```python
from cjm_fasthtml_resources.core.validation import (
    ValidationAction,
    ValidationResult,
    validate_resources_for_job
)
```

#### Functions

```python
def validate_resources_for_job(
    resource_manager,  # ResourceManager instance
    plugin_registry,  # Plugin registry protocol (has get_plugin, load_plugin_config methods)
    get_plugin_resource_requirements,  # Function: (plugin_id, config) -> Dict with requirements
    compare_plugin_resources,  # Function: (config1, config2) -> bool (same resource?)
    get_plugin_resource_identifier,  # Function: (config) -> str (resource ID)
    plugin_id: str,
    plugin_config: Optional[Dict[str, Any]] = None,
    worker_pid: Optional[int] = None,
    worker_type: str = "transcription",
    verbose: bool = False
) -> ValidationResult
    """
    Validate if resources are available to run a job with the specified plugin.
    
    This function is dependency-injected with helper functions to avoid tight coupling
    with specific plugin registry implementations.
    
    Args:
        resource_manager: ResourceManager instance
        plugin_registry: Plugin registry with get_plugin, load_plugin_config methods
        get_plugin_resource_requirements: Function to get plugin requirements
        compare_plugin_resources: Function to compare plugin resources
        get_plugin_resource_identifier: Function to get resource ID
        plugin_id: Unique plugin ID
        plugin_config: Plugin configuration (will load if not provided)
        worker_pid: PID of the worker that will run the job (if known)
        worker_type: Type of worker (e.g., "transcription", "llm", "ollama")
        verbose: Whether to print verbose logging
    
    Returns:
        ValidationResult with action to take
    """
```

#### Classes

```python
class ValidationAction(Enum):
    "Actions that can be taken based on validation results."
```

```python
@dataclass
class ValidationResult:
    "Result of resource validation."
    
    action: ValidationAction
    can_proceed: bool
    message: str
    conflict: Optional[ResourceConflict]
    current_worker: Optional[WorkerState]
    plugin_name_to_reload: Optional[str]  # Plugin name to reload
    new_config: Optional[Dict[str, Any]]  # Config for reload
```
